# Part 4: Using GPU acceleration with PyTorch

In [1]:
# Execute this code block to install dependencies when running on colab
try:
    import torch
except:
    from os.path import exists
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
    cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
    accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-1.0.0-{platform}-linux_x86_64.whl torchvision

try: 
    import torchbearer
except:
    !pip install torchbearer

     |████████████████████████████████| 138 kB 8.6 MB/s 


## Manual use of `.cuda()`

Now the magic of PyTorch comes in. So far, we've only been using the CPU to do computation. When we want to scale to a bigger problem, that won't be feasible for very long.
|
PyTorch makes it really easy to use the GPU for accelerating computation. Consider the following code that computes the element-wise product of two large matrices:

In [2]:
import torch

t1 = torch.randn(1000, 1000)
t2 = torch.randn(1000, 1000)
t3 = t1*t2
print(t3)

tensor([[-6.2223e-01, -1.9143e-01,  3.1051e-02,  ..., -5.5184e-01,
          1.9258e+00,  1.0761e-03],
        [-4.5576e-01, -7.3679e-01,  1.6389e-01,  ...,  9.2868e-01,
          1.9557e-02,  3.7206e-02],
        [-1.0324e+00,  9.7623e-01, -4.2458e-01,  ...,  4.5377e-01,
         -3.6327e-01,  1.0624e+00],
        ...,
        [-4.4243e-02,  3.2484e-01, -6.8879e-01,  ...,  3.6656e-01,
         -2.8853e-01, -1.2673e+00],
        [-2.1522e-02, -4.0590e-01,  3.5220e-01,  ...,  6.1373e-03,
         -1.6867e-02, -1.2106e+00],
        [ 5.1003e-02, -1.8529e-02, -1.4426e+00,  ..., -1.3780e-01,
         -3.7449e-01,  1.0967e+00]])


By sending all the tensors that we are using to the GPU, all the operations on them will also run on the GPU without having to change anything else. If you're running a non-cuda enabled version of PyTorch the following will throw an error; if you have cuda available the following will create the input matrices, copy them to the GPU and perform the multiplication on the GPU itself:

In [3]:
t1 = torch.randn(1000, 1000).cuda()
t2 = torch.randn(1000, 1000).cuda()
t3 = t1*t2
print(t3)

tensor([[-2.2060e-01, -9.5586e-02, -3.4001e-02,  ..., -9.8389e-01,
          2.7769e-01, -7.5569e-02],
        [-4.2502e-01,  3.3133e+00, -2.8268e+00,  ..., -3.2291e-02,
          7.4007e-02, -7.4884e-02],
        [ 4.0262e+00,  1.3911e-01,  1.5188e-02,  ..., -9.1352e-01,
         -1.0702e+00,  1.8311e-01],
        ...,
        [ 2.5558e-01,  9.0250e-02,  8.4933e-01,  ...,  1.9954e-01,
         -8.8629e-01, -4.1099e-01],
        [-1.0293e+00, -7.4615e-01, -1.2436e+00,  ...,  1.1274e-01,
         -1.9519e+00, -2.5249e-02],
        [ 1.1826e+00, -3.1174e-04,  5.2870e-02,  ..., -1.2007e+00,
         -3.2845e-04, -5.9702e+00]], device='cuda:0')


If you're running this workbook in colab, now enable GPU acceleration (`Runtime->Runtime Type` and add a `GPU` in the hardware accelerator pull-down). You'll then need to re-run all cells to this point.

If you were able to run the above with hardware acceleration, the print-out of the result tensor would show that it was an instance of `cuda.FloatTensor` type on the the `(GPU 0)` GPU device. If your wanted to copy the tensor back to the CPU, you would use the `.cpu()` method.

## Writing platform agnostic code

Most of the time you'd like to write code that is device agnostic; that is it will run on a GPU if one is available, and otherwise it would fall back to the CPU. The recommended way to do this is as follows:

In [4]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
t1 = torch.randn(1000, 1000).to(device)
t2 = torch.randn(1000, 1000).to(device)
t3 = t1*t2
print(t3)

tensor([[-0.8770,  0.4554,  0.1962,  ..., -0.4911, -2.1985,  0.0688],
        [-0.1646,  0.5050, -0.1683,  ..., -0.1477,  0.4781, -0.0915],
        [ 0.0586,  0.0584,  0.6362,  ...,  0.0886, -0.8357,  0.0027],
        ...,
        [-0.3248, -0.3492, -0.1722,  ..., -0.6862, -0.4918, -1.9850],
        [ 1.3542,  0.5925, -0.0975,  ..., -0.3493,  1.0075, -2.5992],
        [ 0.3345, -0.0893, -0.0107,  ...,  0.4610,  0.0078, -0.7948]],
       device='cuda:0')


## Accelerating neural net training

If you wanted to accelerate the training of a neural net using raw PyTorch, you would have to copy both the model and the training data to the GPU. Unless you were using a really small dataset like MNIST, you would typically _stream_ the batches of training data to the GPU as you used them in the training loop:

```python
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = BaselineModel(784, 784, 10).to(device)

loss_function = ...
optimiser = ...

for epoch in range(10):
    for data in trainloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimiser.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, labels)
        loss.backward()
        optimiser.step()
```

Using Torchbearer, this becomes much simpler - you just tell the `Trial` to run on the GPU and that's it!:

```python
model = BetterCNN()

loss_function = ...
optimiser = ...

device = "cuda:0" if torch.cuda.is_available() else "cpu"
trial = Trial(model, optimiser, loss_function, metrics=['loss', 'accuracy']).to(device)
trial.with_generators(trainloader)
trial.run(epochs=10)
```


## Multiple GPUs

Using multiple GPUs is beyond the scope of the lab, but if you have multiple cuda devices, they can be referred to by index: `cuda:0`, `cuda:1`, `cuda:2`, etc. You have to be careful not to mix operations on different devices, and would need how to carefully orchestrate moving of data between the devices (which can really slow down your code to the point at which using the CPU would actually be faster).

## Questions

__Answer the following questions (enter the answer in the box below each one):__

__1.__ What features of GPUs allow them to perform computations faster than a typically CPU?

Due to the parallel processing capapbilities of GPU's

__2.__ What is the biggest limiting factor for training large models with current generation GPUs?

Assuming the vRAM of the GPU's